In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load Data

In [ ]:
transit_segments = 

In [ ]:
# Dependent: median speed/95th pct speed, median schedule deviation change
# Independent: seattle street segments variables, acs variables, other variables?
# Models: OLS, NN, Cluster, 

## Create Models